## GPR & LiDAR Figure Map(s)


In [ ]:
import sklearn.metrics as stat
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely.geometry
from geoalchemy2.shape import from_shape
import geoalchemy2.functions as gfunc
import matplotlib.pyplot as plt
import datetime

# some mapping widgets
import ipyleaflet
from ipyleaflet import Map, GeoData, Rectangle, basemaps, LayersControl, basemap_to_tiles, TileLayer, SplitMapControl, Polygon, MagnifyingGlass
import ipywidgets

# database imports
from snowexsql.db import get_db
from snowexsql.data import PointData, LayerData, ImageData, SiteData
from snowexsql.conversions import query_to_geopandas, query_to_pandas, raster_to_rasterio
from snowexsql.db import get_table_attributes

from sqlalchemy.sql import func
from geoalchemy2.types import Raster

import ipyleaflet
from ipyleaflet import Map, Rectangle, basemaps, basemap_to_tiles, TileLayer, SplitMapControl, Polygon
from ipyleaflet import GeoData, LayersControl


from rasterio.plot import show
import shapely.geometry

from geoalchemy2.shape import from_shape
import geoalchemy2.functions as gfunc
import rioxarray
import rasterio
%config InlineBackend.figure_format='retina'

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@db.snowexdata.org/snowex'

engine, session = get_db(db_name)

print('SnowEx Database successfully loaded!')

from snowexsql.data import PointData, LayerData, ImageData, SiteData

# tables available in database ['spatial_ref_sys', 'points', 'layers', 'sites', 'images']


In [ ]:
session.rollback()

In [ ]:
# How to pull out ALL point data from the database that falls within our box
bbox_WSEN = 742000, 4322000, 747000, 4325000 # EPSG 26912?
x1, y1, x2, y2 = bbox_WSEN
polygon = shapely.geometry.Polygon([[x1, y1], [x1, y2], [x2, y2], [x2, y1]]) # used box() before
wkb_element = from_shape(polygon, srid=26912) # which srid is right?

query = session.query(func.ST_AsTiff(func.ST_Clip(func.ST_Union(ImageData.raster, type_=Raster), wkb_element)))
query = query.filter(ImageData.observers == "ASO Inc.")
query = query.filter(ImageData.type == "depth")
query = query.filter(ImageData.date == "2020-02-02")

# Filter the query by bounding box
query = query.filter(gfunc.ST_Intersects(ImageData.raster, wkb_element))

result = query.all()
aso_2020_02_02_rio = raster_to_rasterio(session, result)[0]
aso_2020_02_02_raster = aso_2020_02_02_rio.read(1, masked=True)

# This part takes the `aso_2020_02_02_rio` object we get above and saves as a geoTIFF
filename = "aso_2020_02_02_raster.tif"
with rasterio.open(filename, 'w', **aso_2020_02_02_rio.profile) as f:
    f.write(aso_2020_02_02_rio.read(1), 1)
# Reopen in rioxarray
ASO_snow = rioxarray.open_rasterio(filename, masked=True)
ASO_snow.plot() # probably want to change this but shows the raster on correct coordinate grid

In [ ]:
#import data - initial GPR lines (showing TWT)
GPRLiDAR1 = pd.read_csv('GPRLiDAR1.csv')
GPRLiDAR2 = pd.read_csv('GPRLiDAR2.csv')

GPRLiDAR1 = gpd.GeoDataFrame(GPRLiDAR1)
GPRLiDAR2 = gpd.GeoDataFrame(GPRLiDAR2)
GPRLiDAR1.geometry = gpd.points_from_xy(GPRLiDAR1["ASO_X"], GPRLiDAR1["ASO_Y"])
GPRLiDAR2.geometry = gpd.points_from_xy(GPRLiDAR2["ASO_X"], GPRLiDAR2["ASO_Y"])

GPRLiDAR1.head()


In [ ]:
ax = plt.plot(GPR1['ASO_X'],GPR1['ASO_Y'])
plt.show()

In [ ]:
kwds = { 'label': "SWE [m]", 'orientation': "horizontal"}
ax = GPRLiDAR1.plot(column='SWE', legend=True, cmap='PuBu' , edgecolor = 'none', legend_kwds = kwds)
# Use non-scientific notation for x and y ticks
ax.ticklabel_format(style='plain', useOffset=False)
# Set the various plots x/y labels and title.
ax.set_title('Grand Mesa Jan 28th - Feb 1' )
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')


plt.savefig('GPRLiDAR1SWE.png', dpi=300, bbox_inches='tight')

In [ ]:
kwds = { 'label': "SWE [m]", 'orientation': "horizontal"}
ax = GPRLiDAR2.plot(column='SWE', legend=True, cmap='PuBu' , edgecolor = 'none', legend_kwds = kwds)
# Use non-scientific notation for x and y ticks
ax.ticklabel_format(style='plain', useOffset=False)
# Set the various plots x/y labels and title.
ax.set_title('Grand Mesa  Feb 2 - Feb 9' )
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')


plt.savefig('GPRLiDAR2SWE.png', dpi=300, bbox_inches='tight')


# GPR-LiDAR basic statistics

In [ ]:
GPRLiDAR1.describe().transpose()